## GRADIO WEB APP
Load a ML model and test it with a web interface

Librerie da installare

In [ ]:
#!pip install gradio

Inclusione delle librerie utilizzate

In [ ]:
import gradio as gr
import pandas as pd
import joblib
import numpy as np
import matplotlib
import shap
import matplotlib.pyplot as plt

Variabili di gestione files e parametri del modello

In [ ]:
# Path of the desired model to be tested, can be changed
PATH_MODEL = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\ML_models\Random_Forest\random_forest_2.sav"

# Path of the desired scaler to be tested, can be changed
PATH_SCALER = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\ML_models\Random_Forest\standard_scaler_2.sav"

# Select the type of interface, "Single" for manually inserting a record
# "Multiple" for automatically inserting multiple records from a csv file
type_interface = "Multiple"

Caricamento del modello che normalizza i dati

In [ ]:
loaded_scaler = joblib.load(PATH_SCALER)
print("Scaler caricato:", loaded_scaler)

Caricamento del modello

In [ ]:
loaded_model = joblib.load(PATH_MODEL)
print("Modello caricato:", loaded_model)

In [ ]:
# Load feature names
feature_names = loaded_model.feature_names
print("Feature previste dal modello:", feature_names)

Definizione funzioni

In [ ]:
# Define what to do with data given from the user form
def classify_company(*feature_names):
    # Create a numpy array
    x = np.array([feature_names])

    # Scale it accordingly with the used scaler during the training phase
    scaled_x = loaded_scaler.transform(x)

    # Obtain prediction from model
    prediction = loaded_model.predict_proba(scaled_x)[0]
    prediction_dict = {"Active": float(prediction[0]), "Bankruptcy": float(prediction[1])}

    # Obtain SHAP values plot
    explainer = shap.Explainer(loaded_model)
    shap_values = explainer.shap_values(scaled_x)
    shap.summary_plot(shap_values, scaled_x, show=False, max_display=10,
                      feature_names=loaded_model.feature_names, class_names=["Active", "Bankruptcy"],
                      plot_size=(30, 20))
    fig = plt.gcf()
    plt.close()

    # Return prediction values and shap plot
    return prediction_dict, fig

# Define what to do with data given from the CSV file
def classify_companies(file):
    # Read file as CSV
    input_dataset = pd.read_csv(file.name, usecols=feature_names, sep=r';|,', engine='python', error_bad_lines=False)

    # Remove unused index
    input_dataset.reset_index(drop=True, inplace=True)


    # Force all the feature to be numeric and drop wrong values
    for column in input_dataset.columns:
        input_dataset[column] = pd.to_numeric(input_dataset[column], errors='coerce')
    input_dataset.dropna(inplace=True)


    # Scale it accordingly with the used scaler during the training phase
    x = loaded_scaler.transform(input_dataset)

    # Obtain prediction from model
    predictions = loaded_model.predict_proba(x)
    predictions_bankruptcy = predictions[:, 1]

    # Generate the plot of predictions
    fig = plt.figure(figsize=(15,7))
    plt.hist(predictions_bankruptcy, bins=50)
    plt.xlabel('Probability of bankruptcy', fontsize=25)
    plt.ylabel('Number of records', fontsize=25)
    plt.legend(fontsize=15)
    plt.tick_params(axis='both', labelsize=25, pad=5)

    # Return prediction plot
    return fig

Creazione interfaccia grafica

In [ ]:
# Single record interface
if type_interface == "Single":
    # Input file components
    # For each feature create an optional numeric field
    gradio_inputs = []
    for feature_name in feature_names:
        gradio_inputs.append(gr.inputs.Number(default=0.0, label=feature_name, optional=False))

    # Create output components
    gradio_outputs = [gr.outputs.Label(num_top_classes = 2, label="Prediction probability"), gr.outputs.Plot(type="auto", label="SHAP values")]

    # Create the web app interface
    demo = gr.Interface(
        fn=classify_company,
        inputs=gradio_inputs,
        outputs=gradio_outputs
    )

# Multiple records interface
else:
    # Input file component
    gradio_description = "Il file in formato CSV deve contenere tutti i campi previsti dal modello ed essere possibilmente minore di 350MB."
    gradio_input = gr.inputs.File(file_count="single", type="file", label="CSV file", optional=False)

    # Output file component
    gradio_output = gr.outputs.Plot(type="auto", label="Prediction probabilities")


    # Create the web app interface
    demo = gr.Interface(
        fn=classify_companies,
        inputs=gradio_input,
        outputs=gradio_output,
        description=gradio_description,
        live=True
    )

Esecuzione applicazione web

In [ ]:
demo.launch(show_error=True, inline=False)